In [7]:
import os
import numpy as np 
import pandas as pd 
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupShuffleSplit, GroupKFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score

import xgbfir
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import dtreeviz
from typing import Any, Dict, Union

from yellowbrick import model_selection as ms
from yellowbrick.model_selection import validation_curve

from sklearn import metrics

In [8]:
#Add genotypes
all_animals = pd.read_csv('/home/melissa/RESULTS/FINAL_MODEL/Rat/all_measures_xgboost.csv')
all_animals.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [9]:
all_animals

,Unnamed: 0.1,Animal_ID,Idx,Mot_Disp,Som_Disp,Vis_Disp,Mot_HFD,Som_HFD,Vis_HFD,Mot_Hurst,...,Somatosensory_wpli_beta,Soma_Motor_wpli_beta,Vis_Soma_wpli_beta,Vis_Mot_wpli_beta,Motor_wpli_gamma,Visual_wpli_gamma,Somatosensory_wpli_gamma,Soma_Motor_wpli_gamma,Vis_Soma_wpli_gamma,Vis_Mot_wpli_gamma
0,0,S7088,0,2.924756,2.837509,2.987533,1.246898,1.278877,1.323589,0.529479,...,0.054171,0.038193,0.044965,0.069516,0.059823,0.059157,0.024847,0.013596,0.010702,0.025363
1,1,S7088,1,2.973027,2.756513,3.041175,1.279200,1.283231,1.325623,0.479196,...,0.038475,0.026777,0.034961,0.042132,0.075578,0.091107,0.044866,0.020075,0.020708,0.033455
2,2,S7088,3,3.289658,3.132441,3.246267,1.401297,1.416886,1.421036,0.571518,...,0.037321,0.025463,0.026704,0.041776,0.118101,0.073440,0.045525,0.016685,0.017860,0.032964
3,3,S7088,4,3.030962,2.872915,2.925699,1.373708,1.415605,1.429642,0.586403,...,0.027442,0.025334,0.025045,0.030586,0.109805,0.128597,0.048701,0.027853,0.027767,0.039798
4,4,S7088,5,2.743521,2.643570,2.710767,1.276098,1.298545,1.305473,0.591415,...,0.039938,0.022960,0.026948,0.032765,0.072925,0.052354,0.043338,0.020367,0.022897,0.029437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247242,20049,S7072,34555,2.458509,2.458602,2.367348,1.273711,1.305970,1.312435,0.554247,...,0.020688,0.022312,0.019733,0.073378,0.188963,0.124251,0.038858,0.022101,0.028172,0.073467
247243,20050,S7072,34556,2.592141,2.607698,2.658160,1.209603,1.223521,1.255394,0.536566,...,0.021121,0.022814,0.021229,0.053542,0.192807,0.067435,0.015813,0.010421,0.015343,0.047015
247244,20051,S7072,34557,2.404642,2.429473,2.369505,1.182093,1.208220,1.225206,0.446760,...,0.031027,0.026028,0.019160,0.085103,0.194342,0.107421,0.055217,0.030398,0.021903,0.080197
247245,20052,S7072,34558,2.378175,2.347655,2.334073,1.204421,1.202837,1.224009,0.430673,...,0.013235,0.016505,0.014997,0.049066,0.155751,0.067050,0.025025,0.020540,0.014680,0.053093


In [10]:
wt_ids = ['S7068', 'S7070', 'S7071', 'S7074', 'S7086', 'S7091', 'S7098', 'S7101'] #'S7087',
gap_ids = ['S7063', 'S7064', 'S7069', 'S7072', 'S7075', 'S7076', 'S7088', 'S7092', 'S7094', 'S7096']

In [11]:
def determine_genotype(animal_id, wt_ids, gap_ids):
    if animal_id in wt_ids:
        return 0  # WT
    elif animal_id in gap_ids:
        return 1  # GAP
    else:
        return None  # In case the ID is not found in either list

# Apply the function to each row in the DataFrame and ensure the type is integer
all_animals['Genotype'] = all_animals['Animal_ID'].apply(lambda x: determine_genotype(x, wt_ids, gap_ids)).astype('Int64')

# Move 'Genotype' column to the first position
cols = all_animals.columns.tolist()
cols.insert(0, cols.pop(cols.index('Genotype')))
all_animals = all_animals[cols]

In [15]:
# Combine the two lists and create a list of labels (0 for human_wt and 1 for human_gap)
all_ids = np.unique(all_animals['Animal_ID'].to_list())
labels = [0] * len(wt_ids) + [1] * len(gap_ids)

# Split the combined list into training and test sets, stratifying by the labels
train_ids, test_ids,_, _ = train_test_split(all_ids, labels, test_size=0.2, stratify=labels, random_state= 23)
train_ids

array(['S7071', 'S7075', 'S7076', 'S7088', 'S7101', 'S7064', 'S7083',
       'S7070', 'S7068', 'S7091', 'S7098', 'S7074', 'S7092', 'S7063'],
      dtype='<U5')

In [16]:
test_ids

array(['S7072', 'S7069', 'S7086', 'S7094'], dtype='<U5')

In [9]:
accepted = ['Genotype', 'Animal_ID', 'Idx', 'Mot_CC_Gamma', 'Somatosensory_coh_gamma', 'Vis_CC_Beta', 
'Somatosensory_plv_theta', 'Visual_plv_beta', 'Visual_plv_sigma',
'Mot_CC_Beta', 'Som_CC_Gamma', 'Vis_Mot_pli_sigma', 'Vis_Mot_wpli_sigma',
'Visual_wpli_sigma', 'Vis_Mot_coh_beta', 'Somatosensory_coh_beta',
'Vis_Soma_plv_sigma', 'Somatosensory_wpli_beta', 'Vis_CC_Theta',
'Som_CC_Sigma', 'Soma_Motor_pli_beta', 'Som_CC_Beta', 'Visual_coh_beta',
'Visual_coh_gamma', 'Mot_Disp', 'Soma_Motor_pli_gamma', 'Somatosensory_wpli_gamma',
'Vis_Mot_plv_gamma', 'Somatosensory_pli_beta', 'Soma_Motor_coh_gamma', 'Motor_wpli_gamma',
'Somatosensory_plv_gamma', 'Soma_Motor_plv_beta', 'Somatosensory_plv_delta',
'Visual_plv_gamma', 'Mot_CC_Delta', 'Som_CC_Theta', 'Somatosensory_coh_theta',
'Vis_Mot_pli_gamma', 'Vis_Soma_pli_gamma', 'Vis_Mot_plv_sigma', 'Vis_Soma_plv_theta',
'Motor_wpli_theta', 'Vis_Soma_coh_delta', 'Vis_Soma_plv_gamma', 'Motor_pli_beta',
'Visual_pli_sigma', 'Vis_Soma_plv_delta', 'Motor_pli_theta', 'Vis_CC_Gamma',
'Soma_Motor_plv_gamma', 'Vis_Mot_coh_theta', 'Vis_Soma_coh_theta', 'Motor_coh_gamma',
'Som_Disp', 'Somatosensory_coh_sigma', 'Mot_HFD', 'Vis_Soma_coh_beta', 'Visual_coh_sigma',
'Somatosensory_plv_sigma', 'Mot_CC_Theta', 'Somatosensory_pli_gamma', 'Vis_Soma_plv_beta',
'Visual_pli_gamma', 'Vis_CC_Sigma', 'Vis_Mot_coh_gamma', 'Vis_Disp', 'Soma_Motor_coh_beta',
'Som_Hurst', 'Vis_Hurst', 'Vis_Mot_pli_beta', 'Vis_Mot_pli_theta', 'Som_HFD', 'Motor_pli_sigma',
'Vis_Soma_coh_gamma', 'Vis_Soma_coh_sigma', 'Vis_CC_Delta', 'Vis_Mot_plv_theta', 'Som_CC_Delta',
'Motor_plv_gamma', 'Vis_HFD', 'Somatosensory_plv_beta', 'Mot_Hurst', 'Motor_pli_gamma',
'Vis_Mot_plv_beta']

In [10]:
selected_columns = all_animals[accepted]
selected_columns_clean = selected_columns.dropna()

In [11]:
X_train = selected_columns_clean[selected_columns_clean["Animal_ID"].isin(train_ids)]
X_test = selected_columns_clean[selected_columns_clean["Animal_ID"].isin(test_ids)]
y_train = X_train.iloc[:, 1]
y_test = X_test.iloc[:, 1]

from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy = 'majority')
X_train_res, y_train_res = undersample.fit_resample(X_train, y_train)
X_test_res, y_test_res = undersample.fit_resample(X_test, y_test)

X_train_new = X_train_res.iloc[:, 3:]
X_test_new = X_test_res.iloc[:, 3:]
y_train_new = X_train_res.iloc[:, 0]
y_test_new = X_test_res.iloc[:, 0]

In [12]:
group_by_patient_id = X_train_res.groupby(['Animal_ID'])
groups_by_patient_id_list = np.array(X_train_res['Animal_ID'].values)
groups_by_patient_id_list

array(['S7064', 'S7064', 'S7064', ..., 'S7094', 'S7094', 'S7094'],
      dtype=object)

In [13]:
print(len(X_train_new.loc[X_train_res['Genotype'] == 0]))
print(len(X_train_new.loc[X_train_res['Genotype'] == 1]))
print(len(X_test_new.loc[X_test_res['Genotype'] == 0]))
print(len(X_test_new.loc[X_test_res['Genotype'] == 1]))

69581
78363
22125
20388


In [91]:
X_train_new = X_train_res.iloc[:, 3:]
X_test_new = X_test_res.iloc[:, 3:]
y_train = X_train_res.iloc[:, 0]
y_test = X_test_res.iloc[:, 0]

In [92]:
n_splits = 3
group_kfold = GroupKFold(n_splits = n_splits)
print(group_kfold.get_n_splits(X_train_new, y_train_new, groups = groups_by_patient_id_list))

result = []
y_result = []
for train_idx, val_idx in group_kfold.split(X_train_new, y_train_new, groups = groups_by_patient_id_list):
    train_fold = X_train_new.iloc[train_idx]
    val_fold = X_train_new.iloc[val_idx]
    train_y_fold = y_train_new.iloc[train_idx]
    val_y_fold = y_train_new.iloc[val_idx]
    result.append((train_fold, val_fold))
    y_result.append((train_y_fold, val_y_fold))
    
train_fold_1, val_fold_1 = result[0][0],result[0][1]
train_fold_2, val_fold_2 = result[1][0],result[1][1]
train_fold_3, val_fold_3 = result[2][0],result[2][1]
#train_fold_4, val_fold_4 = result[3][0],result[3][1]
#train_fold_5, val_fold_5 = result[4][0],result[4][1]


y_train_fold_1, y_val_fold_1 = y_result[0][0],y_result[0][1]
y_train_fold_2, y_val_fold_2 = y_result[1][0],y_result[1][1]
y_train_fold_3, y_val_fold_3 = y_result[2][0],y_result[2][1]
#y_train_fold_4, y_val_fold_4 = y_result[3][0],y_result[3][1]
#y_train_fold_5, y_val_fold_5 = y_result[4][0],y_result[4][1]

3


In [93]:
options = {
    'max_depth': hp.quniform('max_depth', 1, 8, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -2, 3),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'reg_alpha': hp.uniform('reg_alpha', 0, 10),
    'reg_lambda': hp.uniform('reg_lambda', 1, 10),
    'gamma': hp.loguniform('gamma', -10, 10),
    'learning_rate': hp.loguniform('learning_rate', -7, 0),
    'n_estimators': hp.choice('n_estimators', range(50, 1001, 50)),
    'scale_pos_weight': hp.uniform('scale_pos_weight', 1, 100),
    'max_delta_step': hp.quniform('max_delta_step', 0, 10, 1),
    'tree_method': 'exact', 
    #'sample_type': hp.choice('sample_type', ['uniform', 'weighted']),
    #'normalize_type': hp.choice('normalize_type', ['tree', 'forest']),
    #'rate_drop': hp.uniform('rate_drop', 0, 1),
    #'skip_drop': hp.uniform('skip_drop', 0, 1),
    'random_state': 42
}


In [ ]:
def hyperparameter_tuning(space: Dict[str, Union[float, int]],
                         X_train: pd.DataFrame, y_train: pd.Series, 
                         X_test: pd.DataFrame, y_test: pd.Series, 
                         early_stopping_rounds: int = 50, 
                         metric: callable = accuracy_score) -> Dict[str, Any]:
    
    '''Perform hyperparameter runing for an XGBoost classifier. 
    
    This function takes a dictionary of hyperparameters, training and test data, and an optional value
    for early stopping rounds, and returns a dictionary with the loss and model resulting from 
    the tuning process. The model is trained using the training data and evaluated on the test 
    data. The loss is computed as the negative of the accuracy score.
    
    space: Dict[str, Union[float, int]]
    A dictionary of hyperparameters for the XGBoost classifier
    
    X_train: pd.DataFrame
    The training data
    
    y_train: pd.Series
    The training target
    
    X_test: pd.Dataframe
    The test data
    
    y_test: pd.Series
    The test target
    
    early_stopping rounds: int, optional 
    The number of early stopping rounds to use. The deault is 50
    
    metric: callable
    Metric to maximise. Default is accuracy
    
    Returns: 
    Dict[str, Any]
        A dictionary with the loss and model resulting from the tuning process. 
        The loss is a float, and the model is an XGBoost classifier'''
    
    int_vals = ['max_depth', 'reg_alpha']
    
    space = {k: (int(val) if k in int_vals else val)
            for k, val in space.items()}
    
    space['early_stopping_rounds'] = early_stopping_rounds
    
    model = xgb.XGBClassifier(**space)
    evaluation = [(X_train, y_train), 
                 (X_test, y_test)]
    model.fit(X_train, y_train, eval_set = evaluation, verbose = False)
    
    score = metrics.roc_auc_score(y_test, model.predict(X_test))
    return {'loss': -score, 'status': STATUS_OK, 'model': model}


options = {'max_depth': hp.quniform('max_depth', 1, 8, 1), #tree
            'min_child_weight': hp.loguniform('min_child_weight', -2, 3),
           'num_leaves': hp.quniform('num_leaves', 20, 250, 10)
            'subsample': hp.uniform('subsample', 0.5, 1), #stochastic
            'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
            'reg_alpha': hp.uniform('reg_alpha', 0, 10), 
            'reg_lambda': hp.uniform('reg_lambda', 1, 10),
            'gamma': hp.loguniform('gamma', -10, 10),
            'learning_rate': hp.loguniform('learning_rate', -7, 0), 
            'random_state': 42
          }


In [95]:
trials = Trials()
best_1 = fmin(fn = lambda space: hyperparameter_tuning(space, X_train = train_fold_1, y_train = y_train_fold_1,
                                                     X_test = val_fold_1, y_test = y_val_fold_1),
            space = options,
            algo = tpe.suggest,
            max_evals = 500,
            trials = trials)

  4%|▎      | 18/500 [00:59<26:21,  3.28s/trial, best loss: -0.4684963901976617]


KeyboardInterrupt: 

In [96]:
trials = Trials()
best_2 = fmin(fn = lambda space: hyperparameter_tuning(space, X_train = train_fold_2, y_train = y_train_fold_2,
                                                     X_test = val_fold_2, y_test = y_val_fold_2),
            space = options,
            algo = tpe.suggest,
            max_evals = 500,
            trials = trials)

  1%|        | 3/500 [00:08<24:24,  2.95s/trial, best loss: -0.2571616904057207]


KeyboardInterrupt: 

In [97]:
trials = Trials()
best_3 = fmin(fn = lambda space: hyperparameter_tuning(space, X_train = train_fold_3, y_train = y_train_fold_3,
                                                     X_test = val_fold_3, y_test = y_val_fold_3),
            space = options,
            algo = tpe.suggest,
            max_evals = 500,
            trials = trials)

 57%|███▍  | 284/500 [12:08<09:14,  2.57s/trial, best loss: -0.6974016357899112]


KeyboardInterrupt: 

In [66]:
best_1 = {'colsample_bytree': 0.5640125501892845,
 'gamma': 886.1540988328427,
 'learning_rate': 0.07054345778305847,
 'max_depth': 4,
 'min_child_weight': 2.637224503774358,
 'reg_alpha': 4.604438973488756,
 'reg_lambda': 3.1664179220499076,
 'subsample': 0.6401529750159931}

In [67]:
best_model = xgb.XGBClassifier(**best_1)
best_model.fit(X_train_new, y_train_new)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5640125501892845, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=886.1540988328427, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.07054345778305847, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=2.637224503774358, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None, ...)

In [68]:
best_model.score(X_test_new, y_test_new)

0.29877402167446915

In [ ]:
def hyperparameter_tuning(space, X, y, n_splits=3):
    # Initialize cross-validation scheme
    cv = StratifiedKFold(n_splits=n_splits)
    
    space['max_depth'] = int(space['max_depth'])
    
    # Store the cross-validated AUC-ROC scores
    cv_scores = []

    for train_idx, test_idx in cv.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        model = xgb.XGBClassifier( **space)
        model.fit(X_train, y_train)

        # Predict probabilities for the positive class (usually column index 1)
        y_pred_proba = model.predict_proba(X_test)[:, 1]

        # Compute AUC-ROC score and append to cv_scores
        score = roc_auc_score(y_test, y_pred_proba)
        cv_scores.append(score)

    # Use the average AUC-ROC score across all folds
    average_cv_score = np.mean(cv_scores)
    
    return {'loss': -average_cv_score, 'status': STATUS_OK}


In [ ]:
def integrated_hyperparameter_tuning(space: Dict[str, Union[float, int]],
                                     X: pd.DataFrame, y: pd.Series, 
                                     n_splits: int = 3, 
                                     early_stopping_rounds: int = 50, 
                                     metric: callable = roc_auc_score) -> Dict[str, Any]:
    '''Perform hyperparameter tuning with cross-validation for an XGBoost classifier.
    
    Parameters:
    space: Dictionary of hyperparameters for the XGBoost classifier.
    X: The complete dataset (features).
    y: The target variable.
    n_splits: Number of folds for cross-validation.
    early_stopping_rounds: Number of early stopping rounds. Default is 50.
    metric: Performance metric function. Default is ROC AUC score.
    
    Returns: 
    Dictionary with the average loss across folds and the model from the last fold.'''

    # Initialize cross-validation scheme
    cv = StratifiedKFold(n_splits=n_splits)
    
    cv_scores = []

    for train_idx, test_idx in cv.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Convert certain hyperparameters to integers if needed
        space = {k: (int(val) if k in ['max_depth', 'reg_alpha'] else val)
                 for k, val in space.items()}
        space['early_stopping_rounds'] = early_stopping_rounds

        model = xgb.XGBClassifier(booster = 'dart',
                                  **space)
        evaluation = [(X_train, y_train), (X_test, y_test)]
        model.fit(X_train, y_train, eval_set=evaluation, verbose=False)

        # Use the metric function provided
        score = metric(y_test, model.predict_proba(X_test)[:, 1])
        cv_scores.append(score)

    average_cv_score = np.mean(cv_scores)
    
    return {'loss': -average_cv_score, 'status': STATUS_OK, 'model': model}


In [ ]:
# Hyperopt settings
trials = Trials()
best = fmin(fn=lambda space: integrated_hyperparameter_tuning(space, X_train_new, y_train, n_splits=3),
            space=options,
            algo=tpe.suggest,
            max_evals=250,
            trials=trials)

In [ ]:
# Save the Trials object to a file
#import pickle
#with open('/home/melissa/RESULTS/FINAL_MODEL/' + 'hyperopt_trials_boruta_rat.pkl', 'wb') as f:
#    pickle.dump(trials, f)


In [ ]:
print("Best hyperparameters:", best)

In [ ]:
print("Best hyperparameters:", best)

# Using the best hyperparameters (example)
best_int = {'colsample_bytree': 0.5572689122012704, 'gamma': 536.9057043679029, 
            'learning_rate': 0.02595304665513714, 'max_depth': 7,
            'min_child_weight': 14.802564724959016,
            'reg_alpha': 8.356830376949699, 'reg_lambda': 3.5657907671974214, 'subsample': 0.6652263592672684}
best_model = xgb.XGBClassifier(**best_int)
best_model.fit(X_train_new, y_train)

In [ ]:
## Test model on each fold: 

In [ ]:
y_train = X_train.iloc[:, 0]
y_test = X_test.iloc[:, 0]

In [ ]:
y_test = X_test.iloc[:, 0]

# Checking for NaN values
nan_in_y_train = y_train.isna().any()
nan_in_y_test = y_test.isna().any()

print("NaN in y_train:", nan_in_y_train)
print("NaN in y_test:", nan_in_y_test)

In [ ]:
X_test.dropna(inplace = True)

In [ ]:
X_test

In [ ]:
X_test.loc[X_test['Genotype'] == 0]

In [ ]:
X_test.iloc[:, 3:]

In [ ]:
# First, get the probability predictions for the positive class
y_pred_proba = best_model.predict_proba(X_test.iloc[:, 3:])[:, 1]

# Now calculate the ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_proba)
roc_auc

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (8,4))

cm = metrics.confusion_matrix(y_train,  best_model.predict(X_train_new))#, normalize = 'true')
disp = metrics.ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = ['DS', 'SE'])
disp.plot(ax = ax, cmap = 'Blues')

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (8,4))

cm_test = metrics.confusion_matrix(y_test, best_model.predict(X_test.iloc[:, 3:]))#, normalize = 'true')
disp = metrics.ConfusionMatrixDisplay(confusion_matrix = cm_test, display_labels = ['DS', 'SE'])
disp.plot(ax = ax, cmap = 'Blues')